<a href="https://colab.research.google.com/github/q890003/ML_Naive-Bayes-Classifier/blob/master/ML_HW2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import sys
import dill
import numpy as np
import math
from itertools import product 
from struct import unpack 

def get_img(img_file_path):
    with open(img_file_path, "rb") as img_file:
        magic, num_imgs, rows, cols = unpack(">IIII", img_file.read(16))

        imgs_mtx = np.zeros((num_imgs, rows, cols), dtype=int)
        for img in range(num_imgs):
            for (row, col) in product(range(rows), range(cols)):
                tmp = unpack(">B", img_file.read(1))
                tmp = int(str(tmp[0]))
                imgs_mtx[img][row,col] = tmp
    return imgs_mtx, num_imgs

def get_label(lbl_file_path):
    with open(lbl_file_path, "rb") as label_file:
        magic, num_itms = unpack(">II", label_file.read(8))
        lbls_mtx = np.zeros((num_itms), dtype=int)
        for img in range(num_itms):
            label = unpack(">B", label_file.read(1))
            label = int(str(label[0]))
            lbls_mtx[img] = label
    return lbls_mtx

label_types = 10
gray_bins = 32
mode = int(sys.argv[1])  #mode = 0 -> Discrete; mode = 1 -> Contineous
# mode = 0    # for test
rows = 28
cols = 28

train_img_pth = "train-images.idx3-ubyte"
train_label_pth = "train-labels.idx1-ubyte"
train_img_mtx, train_num_imgs = get_img(train_img_pth)
train_label_mtx = get_label(train_label_pth)

test_img_pth = "t10k-images.idx3-ubyte"
test_label_pth = "t10k-labels.idx1-ubyte"
test_img_mtx, tests_num_imgs = get_img(test_img_pth)
test_label_mtx = get_label(test_label_pth)
#dill.dump_session('ML_HW2_env.db')

In [23]:
if mode == 0:  #Discrete probability
    #training 
    labels_count = np.zeros((label_types), dtype=float)
    likelyhood = np.zeros((label_types,rows,cols,gray_bins), dtype=float)
    for img in range(train_num_imgs):
        lbl = train_label_mtx[img]
        labels_count[lbl] += 1
        for (row, col) in product(range(rows), range(cols)):
            gray = train_img_mtx[img][row, col] // 8
            likelyhood[lbl][row,col,gray] += 1
    prior = labels_count/train_num_imgs
    # dill.load_session('ML_HW2_env.db')
    #testing
    error_rate = 0 
    for img in range(tests_num_imgs):
        posterior = np.zeros((label_types), dtype=float)
        true_label = test_label_mtx[img]

        for label in range(label_types):
            posterior[label] -= math.log10(labels_count[label])*784
            posterior[label] += math.log10(prior[label])

            for (row, col) in product(range(rows), range(cols)):
                gray = test_img_mtx[img][row, col] //8
                if likelyhood[label][row,col,gray] > 0: 
                    posterior[label] += math.log10(likelyhood[label][row,col,gray])
        
        normalization = np.sum(posterior)
        posterior /= normalization
        
        print("Postirior (in log scale):")
        for i in range(label_types):
            print('{label}: {posterior}'.format(label=i, posterior=posterior[i]))
        print('Prediction: {prediction}, Ans: {ans}'.format(prediction=np.argmin(posterior), ans= true_label))
        if np.argmin(posterior) != true_label:
            error_rate += 1

    #imagination number
    print("Imagination of numbers in Bayesian classifier:")
    for label in range(label_types):
        imagination = np.zeros((rows, cols), dtype=int)
        for (row, col) in product(range(rows), range(cols)):
            imagination[row,col] = np.argmax(likelyhood[label][row,col])//16
        print("{label}: ".format(label=label))
        print(imagination)
    error_rate /= tests_num_imgs
    print(error_rate)

Imagination of numbers in Bayesian classifier:
0: 
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 1 1 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0]
 [0 0

In [24]:
elif mode == 1:      #contineous distribution
    #training
    #dill.load_session('ML_HW2_env.db')
    pixels_mean = np.zeros((label_types, rows, cols), dtype =float)
    pixels_var = np.zeros((label_types, rows, cols), dtype = float)
    labels_count = np.zeros((label_types), dtype=int)

    #pixels mean in each label
    pixels_tmp = np.zeros((label_types, rows, cols), dtype = float)
    for img in range(train_num_imgs):
        lbl = train_label_mtx[img]
        labels_count[lbl] += 1
        pixels_tmp[lbl] += train_img_mtx[img]
    for label in range(label_types):
        pixels_mean[label] = pixels_tmp[label] / labels_count[label]

    #pixels var in each pixel of each label
    for img in range(train_num_imgs):
        lbl = train_label_mtx[img] 
        pixels_var[lbl] += np.square(train_img_mtx[img] - pixels_mean[lbl]) 
    for label in range(label_types):
        pixels_var[label] = pixels_var[label] / labels_count[label]

    #label mean and var
    label_mean = np.mean(labels_count)   
    label_var = np.var(labels_count)

    #testing
    log_two_pi = math.log(2*3.1415926)
    error_rate = 0
    for img in range(tests_num_imgs):
        posterior = np.zeros((label_types), dtype=float)
        for label in range(label_types):
            posterior[label] += -0.5* (log_two_pi+ math.log10(label_var)+(label - label_mean)**2/label_var)
            for (row, col) in product(range(rows), range(cols)):
                    pixel = test_img_mtx[img][row,col]
                    if pixels_var[label][row,col] > 0:
                        pxl_mean = pixels_mean[label][row,col]
                        pxl_var = pixels_var[label][row,col]
                        posterior[label] += -0.5* (log_two_pi+ math.log10(pxl_var)+ (pixel- pxl_mean)**2/pxl_var)

        normalization = np.sum(posterior)
        posterior /= normalization
        true_label = test_label_mtx[img]
        print("Postirior (in log scale):")
        for label in range(label_types):
            print('{label}: {posterior}'.format(label=label, posterior=posterior[label]))
        print('Prediction: {prediction}, ans: {ans}'.format(prediction=np.argmin(posterior), ans= true_label))
        if np.argmin(posterior) != true_label:
            error_rate += 1

    #imagination number
    print("Imagination of numbers in Bayesian classifier:")
    for label in range(label_types):
        imagination = np.zeros((rows, cols), dtype=int)
        for (row, col) in product(range(rows), range(cols)):
            if pixels_mean[label][row, col] > 128:
                imagination[row,col] = 1

        print("{label}: ".format(label=label))
        print(imagination)
    error_rate /= tests_num_imgs
    print("Error_rate", error_rate)

Imagination of numbers in Bayesian classifier:
0: 
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 1 1 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0]
 [0 0